# TensorFlow Tutorial

뉴럴넷과 CNN에 대해 기본적인 강의를 듣고 기초지식을 가지고 있다는 가정하에 진행되는 튜토리얼입니다.

In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '-1' # for using only cpu
import tensorflow as tf
import numpy as np

C:\Users\yangyangii\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Basic Code

In [2]:
a = tf.constant([2],dtype=tf.float32)
b = tf.constant([3],dtype=tf.float32)
c = tf.constant([4],dtype=tf.float32)

op = a*b+c
# op = tf.add(tf.multiply(a,b), c)

sess = tf.Session()
result = sess.run(op)
print(result)
sess.close()

[10.]


tf.constant

In [3]:
# Constant 1-D Tensor populated with value list.
tensor1 = tf.constant([1, 2, 3, 4, 5, 6, 7]) # [1 2 3 4 5 6 7]

# Constant 2-D tensor populated with scalar value -1.
tensor2 = tf.constant(-1.0, shape=[2, 3]) # [[-1. -1. -1.], [-1. -1. -1.]]

sess = tf.Session()
print(sess.run(tensor1))
print(sess.run(tensor2))
print(tensor1)
print(tensor2)
sess.close()

[1 2 3 4 5 6 7]
[[-1. -1. -1.]
 [-1. -1. -1.]]
Tensor("Const_3:0", shape=(7,), dtype=int32)
Tensor("Const_4:0", shape=(2, 3), dtype=float32)


tf.placeholder

In [4]:
input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)
k = tf.constant(2, dtype=tf.float32)
y = x * k

sess = tf.Session()
result = sess.run(y, feed_dict={x:input_data})

print(result)

[ 2.  4.  6.  8. 10.]


In [47]:
input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)
W = tf.Variable([10],dtype=tf.float32)
W = W.assign_add([10.0])
y = W*x

sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

result = sess.run(y, feed_dict={x:input_data})

print(result)

[  20.   40.   60.   80.  100.]


# MNIST w/ Single Layer Perceptron

MNIST Data

In [6]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [7]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
mnist.train.images.shape

(55000, 784)

In [9]:
scipy.misc.toimage(mnist.train.images[999].reshape((28,28)))

In [11]:
mnist.train.labels[999]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [12]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
b = tf.Variable(tf.zeros([10]))

In [13]:
logit = tf.nn.softmax(tf.matmul(images, W) + b)
loss = -tf.reduce_sum(labels*tf.log(logit))

In [14]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_op, feed_dict={images: batch_xs, labels: batch_ys})

Accuracy

In [17]:
correct_prediction = tf.equal(tf.argmax(logit, axis=1), tf.argmax(labels, axis=1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [18]:
print(sess.run(acc, feed_dict={images: mnist.train.images, labels: mnist.train.labels}))

0.89910907


In [19]:
print(sess.run(acc, feed_dict={images: mnist.test.images, labels: mnist.test.labels}))

0.9032


# Practice MLP

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
# W = tf.Variable(tf.zeros([784, 10]))
W1 = tf.Variable(tf.truncated_normal([784, 100], stddev=0.1))
b1 = tf.Variable(tf.zeros([100]))
W2 = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

In [49]:
out1 = tf.nn.relu(tf.matmul(images, W1) + b1)
logit = tf.matmul(out1, W2) + b2
# loss = -tf.reduce_sum(labels*tf.log(logit))
# loss = tf.nn.softmax_cross_entropy_with_logits()
loss = tf.reduce_sum(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)

In [50]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [112]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100, shuffle=True)
    _, batch_loss = sess.run([train_op, loss], feed_dict={images: batch_xs, labels: batch_ys})
print(batch_loss)

146.492


loss

In [113]:
acc = tf.reduce_mean(tf.cast(
    tf.equal(tf.argmax(logit, axis=1), tf.argmax(labels, axis=1)),
    tf.float32))

In [114]:
print(sess.run(acc, feed_dict={images: mnist.train.images, labels: mnist.train.labels}))

0.991273


In [115]:
print(sess.run(acc, feed_dict={images: mnist.test.images, labels: mnist.test.labels}))

0.9769


실습! 성능 높이기: 15분

# MNIST w/ CNN

In [20]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [22]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [23]:
def conv(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [24]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [25]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [26]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])
fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
drop_out = tf.nn.dropout(fc_layer1, keep_prob)

logit = tf.matmul(drop_out, W_fc2) + b_fc2

In [27]:
loss = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
    tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=labels)
)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

BATCH_SIZE = 50
max_epoch = 1
total_batch = int(mnist.train.num_examples/BATCH_SIZE)

# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(max_epoch):
    for i in range(total_batch):
        batch = mnist.train.next_batch(BATCH_SIZE, shuffle=True)
        sess.run(train_op, 
             feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
        )
        train_acc = sess.run(accuracy,
            feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
        )
        print('step %d, training accuracy %g' % (i, train_acc))
        if i % 100 == 0:
            test_acc = sess.run(accuracy, 
                 feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
            )
            print('test accuracy %g' % test_acc)
test_acc = sess.run(accuracy, 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

step 0, training accuracy 0.08
test accuracy 0.068
step 1, training accuracy 0.16
step 2, training accuracy 0.08
step 3, training accuracy 0.14
step 4, training accuracy 0.16
step 5, training accuracy 0.22
step 6, training accuracy 0.14
step 7, training accuracy 0.1
step 8, training accuracy 0.18
step 9, training accuracy 0.22
step 10, training accuracy 0.18
step 11, training accuracy 0.18
step 12, training accuracy 0.18
step 13, training accuracy 0.2
step 14, training accuracy 0.24
step 15, training accuracy 0.3
step 16, training accuracy 0.28
step 17, training accuracy 0.3
step 18, training accuracy 0.36
step 19, training accuracy 0.36
step 20, training accuracy 0.34
step 21, training accuracy 0.3
step 22, training accuracy 0.52
step 23, training accuracy 0.46
step 24, training accuracy 0.3
step 25

KeyboardInterrupt: 

실습! 성능 높이기: 15분, epoch 1000 이내

# Prediction w/ CNN

실습! Prediction 해보기: 10분

In [31]:
input_data = mnist.train.images[999:1000]
print(np.argmax(mnist.train.labels[999]))
scipy.misc.toimage(input_data[0].reshape((28,28)))

2


In [30]:
pred = tf.argmax(logit, 1)
result = sess.run(pred, feed_dict={images: input_data, keep_prob: 1.0})
print('Prediction Result: {}'.format(result[0]))

Prediction Result: 2


숫자 2에 대한 이미지를 예측하고 결과를 보여주는 함수 구현

# Tensorboard w/ CNN

In [1]:
import tensorflow as tf
import numpy as np
import scipy.misc

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def weight_variable(shape, name='weights'):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name='biases'):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

In [4]:
def conv(x, W, name='conv'):
    with tf.name_scope(name) as scope:
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x, name='max_pool'):
    with tf.name_scope(name) as scope:
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [5]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [6]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [7]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])

with tf.name_scope('fc_layer') as scope:
    fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

with tf.name_scope('fc_layer') as scope:
    keep_prob = tf.placeholder(tf.float32)
    drop_out = tf.nn.dropout(fc_layer1, keep_prob)
    logit = tf.matmul(drop_out, W_fc2) + b_fc2

In [8]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.image('images', reshape_images)
tf.summary.scalar('xentropy', loss)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

BATCH_SIZE = 50
max_epoch = 5
total_batch = int(mnist.train.num_examples/BATCH_SIZE)
# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
test_writer = tf.summary.FileWriter('./test', sess.graph)
for epoch in range(max_epoch):
    for i in range(total_batch):
        batch = mnist.train.next_batch(50, shuffle=True)
        sess.run(train_op, 
             feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
        )
        train_acc = sess.run(accuracy,
            feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
        )
        print('step %d, training accuracy %g' % (i, train_acc))
        if i % 100 == 0:
            test_acc, summary = sess.run([accuracy, merged], 
                 feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
            )
            print('test accuracy %g' % test_acc)
            test_writer.add_summary(summary, i)
test_acc, summary = sess.run([accuracy, merged], 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)
test_writer.add_summary(summary, i)

step 0, training accuracy 0.12
test accuracy 0.1019
step 1, training accuracy 0.06
step 2, training accuracy 0.08
step 3, training accuracy 0.06
step 4, training accuracy 0.12
step 5, training accuracy 0.14
step 6, training accuracy 0.2
step 7, training accuracy 0.16
step 8, training accuracy 0.16
step 9, training accuracy 0.22
step 10, training accuracy 0.2
step 11, training accuracy 0.24
step 12, training accuracy 0.2
step 13, training accuracy 0.32
step 14, training accuracy 0.26
step 15, training accuracy 0.3
step 16, training accuracy 0.4
step 17, training accuracy 0.4
step 18, training accuracy 0.56
step 19, training accuracy 0.4
step 20, training accuracy 0.4
step 21, training accuracy 0.56
step 22, training accuracy 0.52
step 23, training accuracy 0.44
step 24, training accuracy 0.5
step 25, training accuracy 0.52
step 26, training accuracy 0.56
step 27, training accuracy 0.44
step 28, training accuracy 0.46
step 29, training accuracy 0.64
step 30, training accuracy 0.5
step 31

step 252, training accuracy 0.96
step 253, training accuracy 0.98
step 254, training accuracy 0.92
step 255, training accuracy 0.92
step 256, training accuracy 0.9
step 257, training accuracy 0.94
step 258, training accuracy 0.94
step 259, training accuracy 0.88
step 260, training accuracy 0.9
step 261, training accuracy 0.92
step 262, training accuracy 1
step 263, training accuracy 0.88
step 264, training accuracy 0.92
step 265, training accuracy 0.94
step 266, training accuracy 0.94
step 267, training accuracy 0.86
step 268, training accuracy 0.94
step 269, training accuracy 0.92
step 270, training accuracy 0.92
step 271, training accuracy 0.92
step 272, training accuracy 0.96
step 273, training accuracy 0.96
step 274, training accuracy 0.94
step 275, training accuracy 0.94
step 276, training accuracy 0.86
step 277, training accuracy 0.96
step 278, training accuracy 0.96
step 279, training accuracy 0.88
step 280, training accuracy 0.86
step 281, training accuracy 0.96
step 282, train

In [9]:
summary

b'\n\x0f\n\x08xentropy\x15~\x8d+>\n\x0f\n\x08accuracy\x158\xf8r?'

Tensorboard를 통해 중요한 요소들을 기록하는 코드를 추가했다.
가장 먼저 loss가 정상적으로 줄어드는지 확인하기 위해 추가하되, loss는 스칼라값이므로 tf.summary.scalar를 통해 기록한다. 이 때 앞에 있는 parameter는 기록할 name이다.
accuracy 또한 iteration마다 어떻게 증가하는지 확인하기 위해 기록한다.

tf.summary.merge_all 함수는 tf.summary를 통해 기록하기 위해 넣은 operation을 모두 병합하여 실행하는 함수다. tf.summary.FileWriter는 기록한 내용을 파일에 쓰는 함수이다. 원하는 폴더를 argument로 넘겨주면 된다. 우리는 test set에 대한 스칼라값만 기록할 것이므로 test 폴더에 기록한다.

sess.run을 통해 test set의 accuracy operation을 실행할 때, merged operation도 함께 실행한다.

파일에 기록한 summary를 TensorBoard로 확인하기 위해서는 해당 폴더에 접근하여,

~\test> tensorboard --logdir=.

위 명령어를 입력하면 된다.
그럼 웹브라우저에서 http://localhost:6006 를 통해 확인할 수 있다.